Загрузка данных

In [ ]:
!rm -Rf my_data
!wget https://github.com/al-filippov/dpo_iis/raw/main/data.tar.gz -O data.tar.gz
!mkdir -p my_data
!tar -xf data.tar.gz -C my_data
!rm data.tar.gz

Установка зависимостей для извлечения текста из doc-файлов

In [ ]:
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr \
flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install textract==1.6.5

Загрузка текста из документов

In [86]:
import os
import textract
import itertools

def myprint(docs):
  count = 100
  doc = list(docs.keys())[0]
  example_value = list(docs.values())[0]
  value = None
  if isinstance(example_value, str):
    example_value = example_value.splitlines(True)
    output_count = min(count, len(example_value))
    value = '\n'.join(example_value[:output_count])
  elif isinstance(example_value, list):
    output_count = min(count, len(example_value))
    value = example_value[:output_count]
  elif isinstance(example_value, dict):
    output_count = min(count, len(example_value))
    value = list(itertools.islice(example_value.items(), output_count))
  print(f'Doc: {doc}\nContent: {value}')

docs = {}
dir_path = "my_data"

for path in os.listdir(dir_path):
    file_path = os.path.join(dir_path, path)
    if os.path.isfile(os.path.join(dir_path, path)):
       docs[path] = textract.process(file_path, encoding = 'unicode_escape').decode("utf8")
myprint(docs)

Doc: tz_02.doc
Content: 



       2.2 Техническое задание





       2.2.1 Общие сведения



      Полное  наименование  системы:    «Автоматизированное  рабочее   место

продавца-консультанта  в салоне  фотоуслуг» в среде «1С: Предприятие 8.1».

      Шифр темы: ДП–УлГТУ– 08080165–05/0978–2010.

      Наименование предприятия заказчика системы, его реквизиты: ООО «УмКо»,

г.Ульяновск;

      Плановые сроки начала и окончания работ: 1 апреля 2009 года — 15  июня

2009 года;

      Сведения об источниках и порядке финансирования работ:  данные  работы

проводятся на безвозмездной основе;

      Порядок оформления  и  предъявления  заказчику  результатов  работ  по

созданию  системы,  ее  частей  и  отдельных  средств:  по  окончанию  работ

заказчику должны быть предъявлены и переданы все  необходимые  дистрибутивы,

набор инструкций и документации.







       2.2.2 Назначение и цели создания системы



      Разработанная система предназначена для автоматизации работы продавца-



Установка библиотеки и модели для обработки текста

In [ ]:
!pip install spacy==3.5.2
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.5.0/ru_core_news_sm-3.5.0.tar.gz

Предобработка текста -- токенизация

In [87]:
import ru_core_news_sm

model = ru_core_news_sm.load()

t_docs = {}

for doc, text in docs.items():
  tokens = model(text)
  t_docs[doc] = []
  for token in tokens:
    str_token = str(token)
    if (len(str_token.strip()) > 0):
      t_docs[doc].append(token)

myprint(t_docs)

Doc: tz_02.doc
Content: [2.2, Техническое, задание, 2.2.1, Общие, сведения, Полное, наименование, системы, :, «, Автоматизированное, рабочее, место, продавца, -, консультанта, в, салоне, фотоуслуг, », в, среде, «, 1С, :, Предприятие, 8.1, », ., Шифр, темы, :, ДП, –, УлГТУ, –, 08080165–05/0978–2010, ., Наименование, предприятия, заказчика, системы, ,, его, реквизиты, :, ООО, «, УмКо, », ,, г., Ульяновск, ;, Плановые, сроки, начала, и, окончания, работ, :, 1, апреля, 2009, года, —, 15, июня, 2009, года, ;, Сведения, об, источниках, и, порядке, финансирования, работ, :, данные, работы, проводятся, на, безвозмездной, основе, ;, Порядок, оформления, и, предъявления, заказчику, результатов, работ, по, созданию, системы, ,, ее, частей]


Предобработка текста -- удаление спец. символов

In [88]:
import re

regex = re.compile('[^a-zA-Zа-яА-Я]')

sp_docs = {}

for doc, tokens in t_docs.items():
  sp_docs[doc] = []
  for token in tokens:
    if len(regex.sub('', str(token))) > 2:
      sp_docs[doc].append(token)

myprint(sp_docs)

Doc: tz_02.doc
Content: [Техническое, задание, Общие, сведения, Полное, наименование, системы, Автоматизированное, рабочее, место, продавца, консультанта, салоне, фотоуслуг, среде, Предприятие, Шифр, темы, УлГТУ, Наименование, предприятия, заказчика, системы, его, реквизиты, ООО, УмКо, Ульяновск, Плановые, сроки, начала, окончания, работ, апреля, года, июня, года, Сведения, источниках, порядке, финансирования, работ, данные, работы, проводятся, безвозмездной, основе, Порядок, оформления, предъявления, заказчику, результатов, работ, созданию, системы, частей, отдельных, средств, окончанию, работ, заказчику, должны, быть, предъявлены, переданы, все, необходимые, дистрибутивы, набор, инструкций, документации, Назначение, цели, создания, системы, Разработанная, система, предназначена, для, автоматизации, работы, продавца-, консультанта, салона, фотоуслуг, принятию, заказов, выполнению, различного, рода, операций, над, ними, ведению, учета, принятых, выполненных, выданных, заказов, Целью]


Предобработка текста -- удаление стоп-слов

In [89]:
stopwords = model.Defaults.stop_words
print(stopwords)

{'больше', 'нам', 'подавно', 'неважно', 'ими', 'чей', 'можем', 'этак', 'снова', 'пора', 'ком', 'нечему', 'нашем', 'должны', 'нынче', 'моей', 'сколько', 'надо', 'некуда', 'сама', 'данному', 'иль', 'щас', 'эта', 'прежде', 'просто', 'самое', 'потомушта', 'наши', 'над', 'раньше', 'наизнанку', 'сей', 'очень', 'едим', 'вправе', 'свой', 'зато', 'нипочем', 'уже', 'поскольку', 'свое', 'другие', 'коли', 'настоящие', 'подобная', 'почему', 'сквозь', 'сразу', 'никем', 'данного', 'отсюда', 'нами', 'такие', 'могла', 'всей', 'ранее', 'одном', 'итак', 'тотчас', 'самому', 'каждое', 'пожалуйста', 'было', 'ныне', 'например', 'негде', 'будьте', 'опять', 'иначе', 'настоящее', 'сможет', 'об', 'менее', 'данная', 'здесь', 'своё', 'моими', 'многому', 'ой', 'посему', 'мой', 'такой', 'наверху', 'моего', 'другая', 'какому', 'внакладе', 'тою', 'отнелиже', 'до', 'всею', 'которые', 'всём', 'настоящая', 'такая', 'них', 'своем', 'ли', 'себя', 'нас', 'данный', 'имела', 'многого', 'он', 'однако', 'н', 'всю', 'одного', 'д

In [90]:
st_docs = {}

for doc, tokens in sp_docs.items():
  st_docs[doc] = []
  for token in tokens:
    if not token.is_stop:
      st_docs[doc].append(token)

myprint(st_docs)

Doc: tz_02.doc
Content: [Техническое, задание, Общие, сведения, Полное, наименование, системы, Автоматизированное, рабочее, место, продавца, консультанта, салоне, фотоуслуг, среде, Предприятие, Шифр, темы, УлГТУ, Наименование, предприятия, заказчика, системы, реквизиты, ООО, УмКо, Ульяновск, Плановые, сроки, окончания, работ, апреля, года, июня, года, Сведения, источниках, порядке, финансирования, работ, работы, проводятся, безвозмездной, основе, Порядок, оформления, предъявления, заказчику, результатов, работ, созданию, системы, частей, отдельных, средств, окончанию, работ, заказчику, предъявлены, переданы, дистрибутивы, набор, инструкций, документации, Назначение, цели, создания, системы, Разработанная, система, предназначена, автоматизации, работы, продавца-, консультанта, салона, фотоуслуг, принятию, заказов, выполнению, различного, рода, операций, ведению, учета, принятых, выполненных, выданных, заказов, Целью, создания, системы, является, повышение, скорости, качества, работы, пр

Предобработка текста -- лемматизация

In [91]:
l_docs = {}

for doc, tokens in st_docs.items():
  l_docs[doc] = []
  for token in tokens:
    l_docs[doc].append(regex.sub('', str(token.lemma_).lower()))

myprint(l_docs)

Doc: tz_02.doc
Content: ['технический', 'задание', 'общий', 'сведение', 'полный', 'наименование', 'система', 'автоматизированный', 'рабочий', 'место', 'продавец', 'консультант', 'салон', 'фотоуслуг', 'среда', 'предприятие', 'шифр', 'тема', 'улгту', 'наименование', 'предприятие', 'заказчик', 'система', 'реквизит', 'ооо', 'умко', 'ульяновск', 'плановый', 'срок', 'окончание', 'работа', 'апрель', 'год', 'июнь', 'год', 'сведение', 'источник', 'порядок', 'финансирование', 'работа', 'работа', 'проводиться', 'безвозмездный', 'основа', 'порядок', 'оформление', 'предъявление', 'заказчик', 'результат', 'работа', 'создание', 'система', 'часть', 'отдельный', 'средство', 'окончание', 'работа', 'заказчик', 'предъявить', 'передать', 'дистрибутив', 'набор', 'инструкция', 'документация', 'назначение', 'цель', 'создание', 'система', 'разработанная', 'система', 'предназначить', 'автоматизация', 'работа', 'продавца', 'консультант', 'салон', 'фотоуслуг', 'принятие', 'заказ', 'выполнение', 'различный', 'род'

Индексирование документов -- создание словаря

In [ ]:
unique_words = set()

for tokens in l_docs.values():
  unique_words |= set(tokens)
  

print(unique_words)

Индексирование документов -- векторизация

In [92]:
ind_docs = {}

for doc in l_docs.keys():
  ind_docs[doc] = dict.fromkeys(unique_words, 0)

myprint(ind_docs)

for doc, tokens in l_docs.items():
  for token in tokens:
    ind_docs[doc][token] += 1

myprint(ind_docs)

Doc: tz_02.doc
Content: [('отдлельных', 0), ('ценообразование', 0), ('вытекать', 0), ('потратить', 0), ('помощь', 0), ('показывать', 0), ('диалог', 0), ('объекту', 0), ('закрываться', 0), ('css', 0), ('spl', 0), ('vatchholder', 0), ('open', 0), ('организуется', 0), ('startsql', 0), ('executeupdate', 0), ('обернуться', 0), ('секретный', 0), ('повышаться', 0), ('мощность', 0), ('серверную', 0), ('прекрасный', 0), ('реселлеров', 0), ('заказчика', 0), ('отладкамай', 0), ('организационно', 0), ('владельцах', 0), ('изменчивый', 0), ('выдать', 0), ('цепь', 0), ('busines', 0), ('правда', 0), ('svga', 0), ('кристина', 0), ('сдача', 0), ('пропущенной', 0), ('таймаут', 0), ('экспериментальный', 0), ('брак', 0), ('экспортный', 0), ('процессорах', 0), ('приступить', 0), ('обучить', 0), ('модератор', 0), ('связь', 0), ('централизованно', 0), ('rules', 0), ('состояниями', 0), ('отображать', 0), ('when', 0), ('сложиться', 0), ('пойти', 0), ('расшифровываться', 0), ('ivanov', 0), ('апплет', 0), ('полно

Индексирование документов -- TF (частота термина/токена)

In [93]:
def compute_tf(tokens_dict, tokens):
    tf_dict = {}
    tokens_count = len(tokens)
    for word, count in tokens_dict.items():
        tf_dict[word] = count / float(tokens_count)
    return tf_dict

tf_docs = {}

for doc, tokens_dict in ind_docs.items():
  tf_docs[doc] = compute_tf(tokens_dict, l_docs[doc])

myprint(tf_docs)

Doc: tz_02.doc
Content: [('отдлельных', 0.0), ('ценообразование', 0.0), ('вытекать', 0.0), ('потратить', 0.0), ('помощь', 0.0), ('показывать', 0.0), ('диалог', 0.0), ('объекту', 0.0), ('закрываться', 0.0), ('css', 0.0), ('spl', 0.0), ('vatchholder', 0.0), ('open', 0.0), ('организуется', 0.0), ('startsql', 0.0), ('executeupdate', 0.0), ('обернуться', 0.0), ('секретный', 0.0), ('повышаться', 0.0), ('мощность', 0.0), ('серверную', 0.0), ('прекрасный', 0.0), ('реселлеров', 0.0), ('заказчика', 0.0), ('отладкамай', 0.0), ('организационно', 0.0), ('владельцах', 0.0), ('изменчивый', 0.0), ('выдать', 0.002109704641350211), ('цепь', 0.0), ('busines', 0.0), ('правда', 0.0), ('svga', 0.0), ('кристина', 0.0), ('сдача', 0.0), ('пропущенной', 0.0), ('таймаут', 0.0), ('экспериментальный', 0.0), ('брак', 0.0), ('экспортный', 0.0), ('процессорах', 0.0), ('приступить', 0.0), ('обучить', 0.0), ('модератор', 0.0), ('связь', 0.0), ('централизованно', 0.0), ('rules', 0.0), ('состояниями', 0.0), ('отображать'

Индексирование документов -- IDF (обратная документная частота термина/токена)

In [94]:
import math
    
docs_count = len(tf_docs.keys())

idf_dict = dict.fromkeys(unique_words, 0)
for doc, tokens_dict in tf_docs.items():
    for token, freq in tokens_dict.items():
        if freq > 0:
            idf_dict[token] += 1

for token, freq in idf_dict.items():
    idf_dict[token] = math.log(docs_count / float(freq))

print(idf_dict)

{'отдлельных': 3.713572066704308, 'ценообразование': 3.713572066704308, 'вытекать': 3.713572066704308, 'потратить': 3.713572066704308, 'помощь': 0.5355182363563621, 'показывать': 1.634130525024472, 'диалог': 2.327277705584417, 'объекту': 3.713572066704308, 'закрываться': 3.0204248861443626, 'css': 3.0204248861443626, 'spl': 3.713572066704308, 'vatchholder': 3.713572066704308, 'open': 2.1041341542702074, 'организуется': 3.713572066704308, 'startsql': 3.713572066704308, 'executeupdate': 3.713572066704308, 'обернуться': 3.713572066704308, 'секретный': 3.713572066704308, 'повышаться': 3.0204248861443626, 'мощность': 1.410986973710262, 'серверную': 3.713572066704308, 'прекрасный': 3.713572066704308, 'реселлеров': 3.713572066704308, 'заказчика': 3.713572066704308, 'отладкамай': 3.713572066704308, 'организационно': 2.327277705584417, 'владельцах': 3.713572066704308, 'изменчивый': 3.713572066704308, 'выдать': 2.614959778036198, 'цепь': 3.713572066704308, 'busines': 3.713572066704308, 'правда':

Индексирование документов -- TF*IDF




In [95]:
tfidf_docs = {}
for doc, tokens_dict in tf_docs.items():
  tfidf_docs[doc] = {}
  for token, freq in tokens_dict.items():
    tfidf_docs[doc][token] = freq * idf_dict[token]

myprint(tfidf_docs)

Doc: tz_02.doc
Content: [('отдлельных', 0.0), ('ценообразование', 0.0), ('вытекать', 0.0), ('потратить', 0.0), ('помощь', 0.0), ('показывать', 0.0), ('диалог', 0.0), ('объекту', 0.0), ('закрываться', 0.0), ('css', 0.0), ('spl', 0.0), ('vatchholder', 0.0), ('open', 0.0), ('организуется', 0.0), ('startsql', 0.0), ('executeupdate', 0.0), ('обернуться', 0.0), ('секретный', 0.0), ('повышаться', 0.0), ('мощность', 0.0), ('серверную', 0.0), ('прекрасный', 0.0), ('реселлеров', 0.0), ('заказчика', 0.0), ('отладкамай', 0.0), ('организационно', 0.0), ('владельцах', 0.0), ('изменчивый', 0.0), ('выдать', 0.0055167927806670835), ('цепь', 0.0), ('busines', 0.0), ('правда', 0.0), ('svga', 0.0), ('кристина', 0.0), ('сдача', 0.0), ('пропущенной', 0.0), ('таймаут', 0.0), ('экспериментальный', 0.0), ('брак', 0.0), ('экспортный', 0.0), ('процессорах', 0.0), ('приступить', 0.0), ('обучить', 0.0), ('модератор', 0.0), ('связь', 0.0), ('централизованно', 0.0), ('rules', 0.0), ('состояниями', 0.0), ('отображать

Кластеризация -- подготовка данных

In [ ]:
!pip install fuzzy-c-means==1.7.0

In [96]:
import numpy as np
from matplotlib import pyplot as plt

data = []

for tokens_dict in tfidf_docs.values():
  data.append(list(tokens_dict.values()))

data = np.array(data)
print(data)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.00649554 0.        ]]


Кластеризация -- разбиение документов по группам с помощью алгоритма FCM (Fuzzy C-Means)

In [97]:
from fcmeans import FCM

clusters = 2
fcm_model = FCM(n_clusters = clusters, m = 1.3)
fcm_model.fit(data)

centers = fcm_model.centers
labels = fcm_model.predict(data)
soft_labels = fcm_model.soft_predict(data)

print(labels)
print(soft_labels)

for cl in range(0, clusters):
  items = []
  for index, label in enumerate(labels):  
    if label == cl:
      items.append(list(tfidf_docs.keys())[index])
  print(f'Cluster {cl + 1}:\n{sorted(items)}')

[0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0
 1 1 1 0]
[[0.50000067 0.49999933]
 [0.49999939 0.50000061]
 [0.5000003  0.4999997 ]
 [0.49999985 0.50000015]
 [0.50000103 0.49999897]
 [0.50000108 0.49999892]
 [0.49999963 0.50000037]
 [0.50000153 0.49999847]
 [0.49999966 0.50000034]
 [0.49999871 0.50000129]
 [0.499998   0.500002  ]
 [0.49999972 0.50000028]
 [0.50000119 0.49999881]
 [0.50000031 0.49999969]
 [0.50000083 0.49999917]
 [0.50000014 0.49999986]
 [0.50000077 0.49999923]
 [0.49999855 0.50000145]
 [0.49999989 0.50000011]
 [0.4999984  0.5000016 ]
 [0.50000098 0.49999902]
 [0.49999869 0.50000131]
 [0.49999957 0.50000043]
 [0.49999971 0.50000029]
 [0.49999821 0.50000179]
 [0.5000014  0.4999986 ]
 [0.50000045 0.49999955]
 [0.50000012 0.49999988]
 [0.50000177 0.49999823]
 [0.50000151 0.49999849]
 [0.49999971 0.50000029]
 [0.49999987 0.50000013]
 [0.49999967 0.50000033]
 [0.50000049 0.49999951]
 [0.50000031 0.49999969]
 [0.49999902 0.50000098]
 [0.50000128 0.

Центры кластеров

In [98]:
def get_max_center_index(term_index, current_center, centers, complete):
  max_index = -1 if current_center in complete else current_center
  max_freq = centers[current_center][term_index]
  for key_center, center in centers.items():
    if key_center == current_center:
      continue
    if key_center in complete:
      continue
    current_freq = center[term_index]
    if current_freq > max_freq:
      max_freq = current_freq
      max_index = key_center
  return max_index


sort_centers = {}
for index, center in enumerate(fcm_model.centers.tolist()):
  my_dict = dict(zip(range(len(center)), center))
  my_dict = dict(sorted(my_dict.items(), key = lambda x:x[1], reverse = True))
  sort_centers[index] = my_dict

max_terms = 10
center_vectors = {}
complete = set()
for key_center in sort_centers.keys():
  center_vectors[key_center] = []
for key_center, center in sort_centers.items():
  for term_index, term in center.items():
    while len(complete) < len(sort_centers.keys()):
      index = get_max_center_index(term_index, key_center, sort_centers, complete)
      if index == -1:
        break
      if len(center_vectors[index]) > max_terms:
        complete.add(index)
        continue
      center_vectors[index].append(list(unique_words)[term_index])
      break

print('Документы:')
for cl in range(0, clusters):
  items = []
  for index, label in enumerate(labels):  
    if label == cl:
      items.append(list(tfidf_docs.keys())[index])
  print(f'Cluster {cl + 1}:\n{sorted(items)}')

print()
print('Центры:')
for index, center in center_vectors.items():
  print(f'Cluster {index + 1}:\n{sorted(center)}')

Документы:
Cluster 1:
['tz_01.doc', 'tz_02.doc', 'tz_03.doc', 'tz_04.doc', 'tz_05.doc', 'tz_06.doc', 'tz_07.doc', 'tz_08.doc', 'tz_09.doc', 'tz_10.doc', 'tz_11.doc', 'tz_12.doc', 'tz_13.doc', 'tz_14.doc', 'tz_15.doc', 'tz_16.doc', 'tz_17.doc', 'tz_18.doc', 'tz_19.doc', 'tz_20.doc']
Cluster 2:
['Архитектура, управляемая модель.doc', 'Введение в проектирование ИС.doc', 'Встроенные операторы SQL.doc', 'Методологии разработки программного обеспечения 2.doc', 'Методологии разработки программного обеспечения.doc', 'Методы композиции и декомпозиции.doc', 'Модели представления данных в СУБД.doc', 'Некоторые особенности проектирования.doc', 'Непроцедурный доступ к данным.doc', 'Процедурное расширение языка SQL.doc', 'Системные объекты базы данных.doc', 'Технология создания распр ИС.doc', 'Требования к проекту.doc', 'Условия целостности БД.doc', 'Характеристики СУБД.doc', 'Этапы разработки проекта1.doc', 'Этапы разработки проекта2.doc', 'Этапы разработки проекта3.doc', 'Этапы разработки проекта4